In [2]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import interpolate
import warnings
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


warnings.filterwarnings('ignore')

import logging
import sys

# Silence Prophet's internal logs
logging.getLogger('cmdstanpy').disabled = True
logging.getLogger().setLevel(logging.WARNING)


# Data Prep

In [3]:
import gdown
import pandas as pd

# Corrected URL of the file
file_id = '1bhS899jbEm_eklwwAvIOr8HEedpqQfdZ'
url = f'https://drive.google.com/uc?id={file_id}&export=download'

# Output file name
output = 'electricity_sales'

# Download the CSV file
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bhS899jbEm_eklwwAvIOr8HEedpqQfdZ&export=download
To: /content/electricity_sales
100%|██████████| 610k/610k [00:00<00:00, 37.6MB/s]


'electricity_sales'

In [4]:
# Load the CSV file into a DataFrame
df = pd.read_excel(output)

In [5]:
df.head()

,Year,Month,State,res_rev,res_sales,res_price,com_rev,com_sales,com_price,ind_rev,ind_sales,ind_price,total_rev,total_sales,total_price
0,2025,1,AK,52477.20,212151.91,24.74,52059.93,239528.29,21.73,23721.38,114883.14,20.65,128258.51,566563.34,22.64
1,2025,1,AL,562365.42,3737790.20,15.05,281333.26,1964636.70,14.32,204513.19,2672376.50,7.65,1048211.90,8374803.40,12.52
2,2025,1,AR,234638.11,2085491.70,11.25,96134.37,959195.75,10.02,109196.13,1691523.70,6.46,439971.21,4736233.30,9.29
3,2025,1,AZ,395240.78,2679716.90,14.75,319933.92,2707643.70,11.82,87363.16,1130490.10,7.73,802625.36,6518654.90,12.31
4,2025,1,CA,2395770.60,7927479.80,30.22,2235178.30,9392828.40,23.80,591968.19,3057323.80,19.36,5232415.70,20434963.00,25.61


In [6]:
df.shape

(2499, 15)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         2499 non-null   int64  
 1   Month        2499 non-null   int64  
 2   State        2499 non-null   object 
 3   res_rev      2499 non-null   float64
 4   res_sales    2499 non-null   float64
 5   res_price    2499 non-null   float64
 6   com_rev      2499 non-null   float64
 7   com_sales    2499 non-null   float64
 8   com_price    2499 non-null   float64
 9   ind_rev      2499 non-null   float64
 10  ind_sales    2499 non-null   float64
 11  ind_price    2499 non-null   float64
 12  total_rev    2499 non-null   float64
 13  total_sales  2499 non-null   float64
 14  total_price  2499 non-null   float64
dtypes: float64(12), int64(2), object(1)
memory usage: 293.0+ KB


In [8]:
df.describe()

,Year,Month,res_rev,res_sales,res_price,com_rev,com_sales,com_price,ind_rev,ind_sales,ind_price,total_rev,total_sales,total_price
count,2499.000000,2499.000000,2.499000e+03,2.499000e+03,2499.000000,2.499000e+03,2.499000e+03,2499.000000,2.499000e+03,2.499000e+03,2499.000000,2.499000e+03,2.499000e+03,2499.000000
mean,2022.551020,6.387755,3.720220e+05,2.429772e+06,15.971749,2.797827e+05,2.274806e+06,12.707215,1.316538e+05,1.659109e+06,9.301224,7.847610e+05,6.374674e+06,12.978463
std,1.161886,3.504730,4.425864e+05,2.700330e+06,6.004195,3.647846e+05,2.496300e+06,5.065467,1.512762e+05,1.912448e+06,4.889632,9.198582e+05,6.700173e+06,5.427315
min,2021.000000,1.000000,2.088093e+04,1.384225e+05,8.830000,2.328483e+04,1.411342e+05,6.430000,5.759400e+02,1.196494e+04,3.820000,6.529020e+04,3.995946e+05,6.370000
25%,2022.000000,3.000000,9.370002e+04,6.443622e+05,12.250000,7.383793e+04,6.481187e+05,9.800000,5.208967e+04,5.043214e+05,6.680000,2.177365e+05,2.161852e+06,9.695000
50%,2023.000000,6.000000,2.533723e+05,1.738707e+06,13.980000,1.881942e+05,1.684141e+06,11.190000,9.497405e+04,1.238099e+06,7.720000,5.567800e+05,4.886749e+06,11.060000
75%,2024.000000,9.000000,4.594088e+05,3.175782e+06,17.220000,3.336815e+05,2.785347e+06,13.240000,1.535826e+05,2.167915e+06,9.310000,9.613908e+05,8.130996e+06,13.725000
max,2025.000000,12.000000,3.264010e+06,2.248142e+07,45.590000,3.322271e+06,1.696502e+07,43.250000,1.372196e+06,1.551005e+07,40.180000,7.748879e+06,5.439652e+07,42.640000


In [9]:
#combining month and year
new_df = df.copy()
new_df['Date'] = pd.to_datetime(new_df[['Year', 'Month']].assign(DAY=1))
new_df['Date'] = new_df['Date'] + pd.offsets.MonthEnd(0)

new_df = new_df.drop(['Year', 'Month'], axis=1)


In [10]:
#total value columns need to be updated since other categories were deleted before reading the excel
new_df['total_rev'] = new_df['res_rev'] + new_df['com_rev'] + new_df['ind_rev']
new_df['total_sales'] = new_df['res_sales'] + new_df['com_sales'] + new_df['ind_sales']
new_df['total_price'] = (new_df['total_rev'] / new_df['total_sales'])*100  # Price = Revenue / Sales (in cents)

In [11]:
new_df.head(100)

,State,res_rev,res_sales,res_price,com_rev,com_sales,com_price,ind_rev,ind_sales,ind_price,total_rev,total_sales,total_price,Date
0,AK,52477.20,212151.91,24.74,52059.93,239528.29,21.73,23721.38,114883.14,20.65,128258.51,566563.34,22.637983,2025-01-31
1,AL,562365.42,3737790.20,15.05,281333.26,1964636.70,14.32,204513.19,2672376.50,7.65,1048211.87,8374803.40,12.516256,2025-01-31
2,AR,234638.11,2085491.70,11.25,96134.37,959195.75,10.02,109196.13,1691523.70,6.46,439968.61,4736211.15,9.289464,2025-01-31
3,AZ,395240.78,2679716.90,14.75,319933.92,2707643.70,11.82,87363.16,1130490.10,7.73,802537.86,6517850.70,12.312922,2025-01-31
4,CA,2395770.60,7927479.80,30.22,2235178.30,9392828.40,23.80,591968.19,3057323.80,19.36,5222917.09,20377632.00,25.630638,2025-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,UT,102896.38,933882.52,11.02,89035.92,1146606.50,7.77,43108.65,694041.11,6.21,235040.95,2774530.13,8.471379,2024-12-31
96,VA,666908.78,4611845.60,14.46,561209.48,6201750.00,9.05,100741.11,1168794.40,8.62,1328859.37,11982390.00,11.090103,2024-12-31
97,VT,50320.11,225778.62,22.29,32239.36,164683.15,19.58,13576.23,112345.80,12.08,96135.70,502807.57,19.119780,2024-12-31
98,WA,495827.80,4190367.80,11.83,285826.28,2699647.10,10.59,104346.56,1701515.70,6.13,886000.64,8591530.60,10.312489,2024-12-31


# Analyzing Sales Growth and Decline from 2023-2024

Since our project focuses on forecasting electricity sales for selective states, we need to group it by states and the sectors.

States are selected based on recent trends (2023–2024) by analyzing sales growth, volatility, and sector dependence. This helped identify high-growth, unstable, or sector-dominant states for deeper forecasting.

In [12]:
#Selecting states based on the most sales growth:
new_df['year'] = new_df['Date'].dt.year
sales_by_year = (
    new_df[new_df['year'].isin([2023, 2024])]
    .groupby(['State', 'year'])['total_sales']
    .sum()
    .unstack()
    .dropna()
)

#growth percentage
sales_by_year['growth_%'] = 100 * (sales_by_year[2024] - sales_by_year[2023]) / sales_by_year[2023]

# Top 5 growing Sales
top_growth = sales_by_year.sort_values('growth_%', ascending=False).head(5)
print("Top 5 growth states:\n", top_growth)

# Declining Sales
bottom_growth = sales_by_year.sort_values('growth_%').head(5)
print("Bottom 5 states by growth:\n", bottom_growth)



Top 5 growth states:
 year          2023         2024  growth_%
State                                    
NM     28347490.01  30344453.71  7.044587
NV     38242174.12  40625854.29  6.233119
OK     68978840.40  72720381.90  5.424187
AZ     85908732.20  90293493.50  5.103976
ND     28202179.26  29627583.64  5.054235
Bottom 5 states by growth:
 year          2023         2024  growth_%
State                                    
MA     49732618.01  48994259.08 -1.484657
MN     66197771.30  65338393.90 -1.298197
ME     11336030.02  11244146.74 -0.810542
HI      8927251.99   8899432.63 -0.311623
SD     13505999.02  13488669.19 -0.128312


**Top 5 States by Sales Growth (2023 → 2024):**
- New Mexico (NM)
- Nevada (NV)
- Oklahoma (OK)
- Arizona (AZ)
- North Dakota (ND)

**Bottom 5 States by Sales Growth (2023 → 2024):**
- Massachusetts (MA)
- Minnesota (MN)
- Maine (ME)
- Hawaii (HI)
- South Dakota (SD)

**Why this selection is valid:**
These states were chosen based on their year-over-year total electricity sales growth, ensuring a balanced analysis of both high-performing and underperforming regions. This approach highlights where demand is rising rapidly and where it may be stagnating or declining, providing valuable insights for future forecasting and resource planning.


# Visualizations

In [13]:
#final df with selected states
final_states = ['NM', 'NV', 'OK', 'AZ', 'ND', 'MA', 'MN', 'ME', 'HI', 'SD']
df_wStates = new_df[new_df['State'].isin(final_states)].copy()

In [14]:
# df_wStates['Date'] = df_wStates['Date'].dt.to_period('M').dt.to_timestamp()

grouped_final_df = df_wStates[df_wStates['Date'] < '2025-01-01'].groupby(['State', 'Date'], as_index=False).agg({
    'res_sales': 'sum',
    'com_sales': 'sum',
    'ind_sales': 'sum',
    'total_sales': 'sum',
})
grouped_final_df

,State,Date,res_sales,com_sales,ind_sales,total_sales
0,AZ,2021-01-31,2591048.60,2122618.30,1141448.20,5855115.10
1,AZ,2021-02-28,2015271.00,1931320.50,1058099.00,5004690.50
2,AZ,2021-03-31,2150052.60,2112299.50,1127293.30,5389645.40
3,AZ,2021-04-30,2399872.60,2282122.40,1099530.50,5781525.50
4,AZ,2021-05-31,3115253.30,2548114.40,1177562.80,6840930.50
...,...,...,...,...,...,...
475,SD,2024-08-31,445337.07,473820.48,304585.04,1223742.59
476,SD,2024-09-30,387652.90,433731.10,294795.92,1116179.92
477,SD,2024-10-31,345892.82,414735.56,266795.19,1027423.57
478,SD,2024-11-30,380255.32,402848.00,274881.63,1057984.95


1. Total Sales over time from all the Selected States

In [15]:
import plotly.express as px
# px.line(grouped_final_df, x="Date", y="total_sales", color="State", title="Total Sales Over Time", line_shape='spline' )
total_fig = px.line(grouped_final_df, x="Date", y="total_sales", color="State",
              title="Total Sales Over Time", line_shape='spline')

# Add vertical lines for each year-end
year_ends = ["2021-12-31", "2022-12-31", "2023-12-31"]

for date in year_ends:
    total_fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="gray")

total_fig.update_layout(
    width=1100,
    height=550
)

total_fig.show()

2. What it shows: How each sector (residential, commercial, industrial) behaves across months

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# List states
states = grouped_final_df['State'].unique()

# starting with AZ
initial_state = states[0]
state_df = grouped_final_df[grouped_final_df['State'] == initial_state]

fig = px.line(
    state_df,
    x='Date',
    y=['res_sales', 'com_sales', 'ind_sales'],
    labels={'value': 'Sales (MWh)', 'variable': 'Sector'},
)

year_ends =  ["2021-12-31", "2022-12-31", "2023-12-31"]
for date in year_ends:
    fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="gray")


##code for interaction features
fig.update_layout(

    title=f'Seasonal Usage Trends by Sector in {initial_state}',
    updatemenus=[{
        'buttons': [
            {
                'label': state,
                'method': 'update',
                'args': [
                    {
                        'y': [
                            grouped_final_df[grouped_final_df['State'] == state]['res_sales'], #shows residensial, commercial, and industrial sales for each state
                            grouped_final_df[grouped_final_df['State'] == state]['com_sales'],
                            grouped_final_df[grouped_final_df['State'] == state]['ind_sales'],
                        ]
                    },
                    {'title': f'Seasonal Usage Trends by Sector in {state}'}
                ]
            } for state in states
        ],
        'direction': 'down',
        'showactive': True,
        'x': 1.1,
        'xanchor': 'left',
        'y': 1.1,
        'yanchor': 'top'
    }]
)

fig.show()


3. What it shows: What % of each state’s electricity is used by each sector — are they residential-heavy? Industrial? Balanced?

In [ ]:
# Aggregate sector shares per state
df_sector_pct = (
    grouped_final_df.groupby('State')[['res_sales', 'com_sales', 'ind_sales']].sum() #without the time
)
df_sector_pct = df_sector_pct.div(df_sector_pct.sum(axis=1), axis=0).reset_index()

df_melted = df_sector_pct.melt(id_vars='State', var_name='Sector', value_name='Share')

fig = px.bar(
    df_melted,
    x='State',
    y='Share',
    color='Sector',
    title='Sector Dependence Per State',
    text_auto='.0%',
)
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'})
fig.show()


# Time-Series Forecast for 2025-2026

In [16]:
grouped_final_df

,State,Date,res_sales,com_sales,ind_sales,total_sales
0,AZ,2021-01-31,2591048.60,2122618.30,1141448.20,5855115.10
1,AZ,2021-02-28,2015271.00,1931320.50,1058099.00,5004690.50
2,AZ,2021-03-31,2150052.60,2112299.50,1127293.30,5389645.40
3,AZ,2021-04-30,2399872.60,2282122.40,1099530.50,5781525.50
4,AZ,2021-05-31,3115253.30,2548114.40,1177562.80,6840930.50
...,...,...,...,...,...,...
475,SD,2024-08-31,445337.07,473820.48,304585.04,1223742.59
476,SD,2024-09-30,387652.90,433731.10,294795.92,1116179.92
477,SD,2024-10-31,345892.82,414735.56,266795.19,1027423.57
478,SD,2024-11-30,380255.32,402848.00,274881.63,1057984.95


In [17]:
from prophet import Prophet
forecast_states = ['NM', 'NV', 'OK', 'AZ', 'ND', 'MA', 'MN', 'ME', 'HI', 'SD']
sector_cols = ['res_sales', 'com_sales', 'ind_sales']

all_forecasts = []

for state in forecast_states: # for each state
    for sector in sector_cols: #for each sector
        df_sub = grouped_final_df[grouped_final_df['State'] == state][['Date', sector]].copy() #subset of that specific sector
        df_sub.rename(columns={'Date': 'ds', sector: 'y'}, inplace=True)

        # skipping sectors with no variation
        if df_sub['y'].nunique() < 2:
            continue

        # fit Prophet model
        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(df_sub) #fit the subset of the sector for the state

        future = model.make_future_dataframe(periods=24, freq='MS') # future prediction of next 2 years 2025-2026
        forecast = model.predict(future)

        # Keep only needed cols
        last_date = df_sub['ds'].max() # last date will be dec 2024
        forecast_trimmed = forecast[forecast['ds'] > last_date][['ds', 'yhat']].copy() #trimming predicted sale values for 2025-2026
        forecast_trimmed['State'] = state
        forecast_trimmed['Sector'] = sector
        all_forecasts.append(forecast_trimmed)

In [18]:
sector_forecast_df = pd.concat(all_forecasts).reset_index(drop=True)
sector_forecast_df

,ds,yhat,State,Sector
0,2025-01-01,566001.416306,NM,res_sales
1,2025-02-01,752502.616685,NM,res_sales
2,2025-03-01,575972.445088,NM,res_sales
3,2025-04-01,486535.260223,NM,res_sales
4,2025-05-01,484353.980229,NM,res_sales
...,...,...,...,...
715,2026-08-01,300086.574247,SD,ind_sales
716,2026-09-01,313783.490008,SD,ind_sales
717,2026-10-01,298868.604356,SD,ind_sales
718,2026-11-01,264551.138228,SD,ind_sales


### Final Forecasted Sales for 2025-2026

In [19]:
cleanForecast_df = sector_forecast_df.pivot_table(
    index=['ds', 'State'],
    columns='Sector', #seperate cols for each state
    values='yhat'
).reset_index()

In [20]:
cleanForecast_df

Sector,ds,State,com_sales,ind_sales,res_sales
0,2025-01-01,AZ,2.600252e+06,1.089828e+06,2.359069e+06
1,2025-01-01,HI,2.332465e+05,2.969423e+05,2.170097e+05
2,2025-01-01,MA,2.071230e+06,5.508637e+05,1.853867e+06
3,2025-01-01,ME,3.617408e+05,1.979330e+05,4.708741e+05
4,2025-01-01,MN,1.974628e+06,1.703046e+06,2.353818e+06
...,...,...,...,...,...
235,2026-12-01,ND,1.150356e+06,1.325878e+06,3.720157e+05
236,2026-12-01,NM,7.807378e+05,1.558098e+06,5.373019e+05
237,2026-12-01,NV,1.026060e+06,1.061846e+06,8.774040e+05
238,2026-12-01,OK,2.446706e+06,1.874102e+06,1.433687e+06


### Forecast Accuracy

In [21]:
#Jan predictions
jan_2025_forecast = cleanForecast_df[cleanForecast_df['ds'] == '2025-01-01']
jan_2025_forecast = jan_2025_forecast.melt(id_vars=["State", "ds"],
                                         value_vars=["res_sales", "com_sales", "ind_sales"],
                                         var_name="Sector", value_name="yhat")


#Actual Jan Data
jan_2025_actuals = df_wStates[(df_wStates['Date'] == '2025-01-31') & (df_wStates['State'].isin(forecast_states))] # filter date and states the forcast was trained on
jan_2025_actuals = jan_2025_actuals.melt(id_vars=["State", "Date"],
                                         value_vars=["res_sales", "com_sales", "ind_sales"],
                                         var_name="Sector", value_name="Actual")

#need to match the format of the 2 datasets
# jan_2025_actuals["Sector"] = jan_2025_actuals["Sector"]
jan_2025_actuals['Date'] = pd.to_datetime(jan_2025_actuals['Date']).dt.to_period('M').dt.to_timestamp()


Forecast accuracy is validated using January 2025 actual data, comparing it against the model's prediction for that month. This approach offers short-term validation

In [22]:
jan_2025_forecast

,State,ds,Sector,yhat
0,AZ,2025-01-01,res_sales,2.359069e+06
1,HI,2025-01-01,res_sales,2.170097e+05
2,MA,2025-01-01,res_sales,1.853867e+06
3,ME,2025-01-01,res_sales,4.708741e+05
4,MN,2025-01-01,res_sales,2.353818e+06
5,ND,2025-01-01,res_sales,6.772310e+05
6,NM,2025-01-01,res_sales,5.660014e+05
7,NV,2025-01-01,res_sales,1.010889e+06
8,OK,2025-01-01,res_sales,2.010532e+06
9,SD,2025-01-01,res_sales,5.757261e+05


In [23]:
jan_2025_actuals

,State,Date,Sector,Actual
0,AZ,2025-01-01,res_sales,2679716.90
1,HI,2025-01-01,res_sales,240046.47
2,MA,2025-01-01,res_sales,2020388.00
3,ME,2025-01-01,res_sales,463639.03
4,MN,2025-01-01,res_sales,2390251.00
5,ND,2025-01-01,res_sales,648054.78
6,NM,2025-01-01,res_sales,681870.51
7,NV,2025-01-01,res_sales,974877.80
8,OK,2025-01-01,res_sales,2582921.20
9,SD,2025-01-01,res_sales,595505.67


In [24]:
jan_compare = pd.merge(jan_2025_forecast, jan_2025_actuals,   ##combine the actual and forecast values to compare
                       left_on=["State", "Sector", "ds"],
                       right_on=["State", "Sector", "Date"])

# Calculate metrics
mae = mean_absolute_error(jan_compare['Actual'], jan_compare['yhat'])
rmse = np.sqrt(mean_squared_error(jan_compare['Actual'], jan_compare['yhat']))
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Mean Absolute Error (MAE): 78143.80
Root Mean Squared Error (RMSE): 140353.90


Mean Absolute Error (MAE): ~78143.80
→ On average, our predictions were off by around 78,000 units.

Root Mean Squared Error (RMSE) ~140353.90 → Suggests a few large errors are inflating the MAE.

In [ ]:
jan_compare["ape"] = abs(jan_compare["Actual"] - jan_compare["yhat"]) / jan_compare["Actual"]
mape = jan_compare["ape"].mean() * 100
print(f"📈 MAPE: {mape:.2f}%") #2 decimal points

📈 MAPE: 5.71%


It means that, on average, **our predictions were within 5.71% of the actual values.**
In real-world forecasting, anything below 10% is considered highly accurate

# Analysis for 2025-2026 Sales

In [25]:
import plotly.graph_objects as go

# === Prepare data ===
actuals_df = grouped_final_df[['Date', 'State'] + sector_cols].copy()
actuals_df.rename(columns={col: f'{col}_actual' for col in sector_cols}, inplace=True)

forecasts_df = cleanForecast_df.rename(columns={'ds': 'Date'}).copy()
forecasts_df.rename(columns={col: f'{col}_forecast' for col in sector_cols if col in forecasts_df.columns}, inplace=True)

combined_df = pd.merge(actuals_df, forecasts_df, on=['Date', 'State'], how='outer')
combined_df = combined_df.sort_values(by=['State', 'Date'])

states = combined_df['State'].dropna().unique()
traces = []
buttons = []
visibility_matrix = []

# === Build Traces for Each State ===
for i, state in enumerate(states):
    df = combined_df[combined_df['State'] == state]
    state_traces = []

    for sector in sector_cols:
        # Actual
        state_traces.append(go.Scatter(
            x=df['Date'],
            y=df[f'{sector}_actual'],
            mode='lines',
            name=f'{state} - {sector} (actual)',
            line=dict(dash='solid'),
            visible=(i == 0)  # Only initial state visible
        ))

        # Forecast
        state_traces.append(go.Scatter(
            x=df['Date'],
            y=df[f'{sector}_forecast'],
            mode='lines',
            name=f'{state} - {sector} (forecast)',
            line=dict(dash='dot'),
            visible=(i == 0)
        ))

    traces.extend(state_traces)

# === Build Visibility List for Dropdown Buttons ===
n_traces_per_state = len(sector_cols) * 2

for i, state in enumerate(states):
    vis = [False] * len(traces)
    start = i * n_traces_per_state
    vis[start:start + n_traces_per_state] = [True] * n_traces_per_state

    buttons.append(dict(
        label=state,
        method='update',
        args=[{'visible': vis},
              {'title': f'Sector-wise Energy Usage and Forecast – {state}'}]
    ))

# === Build Final Figure ===
fig = go.Figure(data=traces)

fig.update_layout(
    title=f'Sector-wise Energy Usage and Forecast – {states[0]}',
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'x': 1.1,
        'y': 1.1,
        'xanchor': 'left',
        'yanchor': 'top'
    }]
)

# Add year-end markers
year_ends = ["2021-12-31", "2022-12-31", "2023-12-31"]
for date in year_ends:
    fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="gray")

fig.show()


In [ ]:
# Aggregate sector shares per state
sector_pct_25 = (
    cleanForecast_df.groupby('State')[['res_sales', 'com_sales', 'ind_sales']].sum() #without the time
)
sector_pct_25 = sector_pct_25.div(sector_pct_25.sum(axis=1), axis=0).reset_index()

melted_25 = sector_pct_25.melt(id_vars='State', var_name='Sector', value_name='Share')

fig3 = px.bar(
    melted_25,
    x='State',
    y='Share',
    color='Sector',
    title='Sector Dependence Per State 2025-2026',
    text_auto='.0%',
)
fig3.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'})
fig3.show()